In [1]:
%cd /home/slav/ai/claim-rank
!export PYTHONPATH=.
%load_ext autoreload
%autoreload 2

/home/slav/ai/claim-rank


In [2]:
import numpy as np
np.random.seed(42) # ! before importing keras!

In [3]:
from src.data.debates import read_debates, Debate

In [4]:
from src.features.feature_sets import get_cb_pipeline

In [5]:
from keras.layers import Input, Embedding, Conv1D, Dense
from keras.models import Model, clone_model
from keras.layers import Input, Dense, Dropout
from keras import optimizers
from src.stats.rank_metrics import average_precision, precision_at_n, accuracy_rounded
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing.data import MinMaxScaler

Using TensorFlow backend.


In [6]:
train_sentences = read_debates(Debate.FIRST) + read_debates(Debate.VP)
val_sentences = read_debates(Debate.SECOND)
test_sentences = read_debates(Debate.THIRD)

pipeline = get_cb_pipeline(train=train_sentences)
X_train = pipeline.fit_transform(train_sentences)
X_val = pipeline.fit_transform(val_sentences)
X_test = pipeline.transform(test_sentences)

y_train_pf = np.array([int(s.labels[5]) for s in train_sentences])
y_train_all = np.array([1 if s.label>0 else 0 for s in train_sentences])

y_val_pf = np.array([int(s.labels[5]) for s in val_sentences])
y_val_all = np.array([1 if s.label>0 else 0 for s in val_sentences])

y_test_pf = np.array([int(s.labels[5]) for s in test_sentences])
y_test_all = np.array([1 if s.label>0 else 0 for s in test_sentences])


y_train_0 = np.array([int(s.labels[0]) for s in train_sentences])
y_train_1 = np.array([int(s.labels[1]) for s in train_sentences])
y_train_2 = np.array([int(s.labels[2]) for s in train_sentences])
y_train_3 = np.array([int(s.labels[3]) for s in train_sentences])
y_train_4 = np.array([int(s.labels[4]) for s in train_sentences])
y_train_5 = np.array([int(s.labels[5]) for s in train_sentences])
y_train_6 = np.array([int(s.labels[6]) for s in train_sentences])

['tokens_num', 'text_len', 'bag_tfidf', 'pos']


In [21]:
print(X_train.shape)
print(train_sentences[103].labels)
print(train_sentences[103].label)
print(X_train.shape[1])
print(np.array(y_train_all).shape)
print(y_val_pf[13])
print(y_val_all[13])

(2761, 1363)
['0', '0', '1', '0', '0', '0', '0', '0', '0']
1
1363
(2761,)
1
1


In [22]:
features_count = X_train.shape[1]

In [19]:
def create_model_old(out_count):
    input_layer = Input(shape=(X_train.shape[1],))
    x = Dense(200, activation='relu')(input_layer)
    x = Dense(50, activation='relu')(x)
    
    outputs = list(map(lambda _: Dense(1, activation='sigmoid')(x), range(out_count)))

    model = Model(inputs=[input_layer], outputs=outputs)

    model.compile(optimizer='rmsprop', 
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model;
    

In [20]:
def create_model(out_count):
    input_layer = Input(shape=(X_train.shape[1],))
    x = Dense(200, activation='relu')(input_layer)
    x = Dense(50, activation='relu')(x)
    
    outputs = list(map(lambda _: Dense(1, activation='sigmoid')(x), range(out_count)))

    model = Model(inputs=[input_layer], outputs=outputs)
    
    sgd = optimizers.SGD(lr=0.01)
                         
    model.compile(optimizer=sgd, 
                  loss='mean_squared_logarithmic_error',
                  metrics=['accuracy'])
    return model;
    

In [33]:
def create_baseline_model(out_count):
    input_layer = Input(shape=(features_count,))
    x = Dense(features_count, kernel_initializer='normal', activation='relu')(input_layer)
    
    outputs = list(map(lambda _: Dense(1, kernel_initializer='normal', activation='sigmoid')(x), range(out_count)))

    model = Model(inputs=[input_layer], outputs=outputs)
                         
    model.compile(optimizer='adam', 
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model;

In [34]:
def run_single_model(train_target, label_func, iters = 20, epochs=5):
    model = create_baseline_model(1);

    best_model = None;
    best_av_p = 0.;
    best_epoch = -1;

    for epoch in range(iters):
        
        model.fit(X_train, y=[train_target], epochs=epochs, verbose=0, batch_size=50)

        predicted_val = model.predict(X_val)
        
        for pred, sentence in zip(predicted_val, val_sentences):
            sentence.pred = pred[0]

        av_p = average_precision(val_sentences,1, label_func)
        
        print(str(epoch) + ':' + str(av_p) + ' ', end='')
        
        if (av_p > best_av_p):
            best_model = clone_model(model);
            best_av_p = av_p;
            best_epoch = epoch;
    
    print()

    print(best_av_p)
    print((best_epoch + 1) * epochs)

    predicted_val = best_model.predict(X_test)
    for pred, sentence in zip(predicted_val, test_sentences):
        sentence.pred = pred[0]

    print(average_precision(test_sentences,1,label_func))
    
    return [best_model, best_av_p, best_epoch]
    

In [13]:
def run_multiple_model(train_targets, iters = 20, epochs=5):
    model = create_model_old(len(train_targets));

    best_model = None;
    best_av_p = 0.;
    best_epoch = -1;

    best_model_all = None;
    best_av_p_all = 0.;
    best_epoch_all = -1;

    for epoch in range(iters):
#         print(str(epoch) + ' ', end='')

        model.fit(X_train, y=train_targets, epochs=epochs, verbose=0, batch_size=550)

        predicted_val = model.predict(X_val)
        
        for pred, sentence in zip(predicted_val[1], val_sentences):
            sentence.pred = pred[0]

        av_p = average_precision(val_sentences,1, lambda x: x.labels[5])
        if (av_p > best_av_p):
            best_model = clone_model(model);
            best_av_p = av_p;
            best_epoch = epoch;

        for pred, sentence in zip(predicted_val[0], val_sentences):
            sentence.pred = pred[0]

        av_p_all = average_precision(val_sentences,1)
        if (av_p_all > best_av_p_all):
            best_model_all = clone_model(model);
            best_av_p_all = av_p_all;
            best_epoch_all = epoch;
    
#     print()
    
#     print('All:')
#     print(best_av_p_all)
#     print((best_epoch_all+ 1) * epochs)

#     predicted_val = best_model_all.predict(X_test)
#     for pred, sentence in zip(predicted_val[0], test_sentences):
#         sentence.pred = pred[0]

#     print(average_precision(test_sentences,1))

#     print('Pf:')
#     print(best_av_p)
#     print((best_epoch + 1) * epochs)

#     predicted_val = best_model.predict(X_test)
#     for pred, sentence in zip(predicted_val[1], test_sentences):
#         sentence.pred = pred[0]

#     print(average_precision(test_sentences,1,lambda x: x.labels[5]))
    
    return [[best_model, best_av_p, best_epoch], [best_model_all, best_av_p_all, best_epoch_all]]
    

In [14]:
def run_multiple_times_single(train_targets, label_func, iters = 100):
    best_model = None;
    best_av_p = 0.;
    best_epoch = -1;
    
    for i in range(iters):
        model, av_p, epoch = run_single_model(train_targets, label_func)
        print(str(i) + ':' + str(av_p) + ' ', end='')
        if (av_p > best_av_p):
            best_model = clone_model(model);
            best_av_p = av_p;
            best_epoch = epoch;
    print()

    print(best_av_p)
    print(best_epoch )

    predicted_val = best_model.predict(X_test)
    for pred, sentence in zip(predicted_val, test_sentences):
        sentence.pred = pred[0]

    print(average_precision(test_sentences,1,label_func))
    

In [15]:
def run_multiple_times_multiple(train_targets, iters = 10):
    best_model = None;
    best_av_p = 0.;
    best_epoch = -1;
    
    
    best_model_all = None;
    best_av_p_all = 0.;
    best_epoch_all = -1;

    for i in range(iters):
        single_p, all_p = run_multiple_model(train_targets)
        model, av_p, epoch = single_p
        model_all, av_p_all, epoch_all = all_p
        
        print(str(i) + ':' + str(av_p) + ' ', end='')
        print(str(i) + ':' + str(av_p_all) + ' ', end='')
        
        if (av_p > best_av_p):
            best_model = clone_model(model);
            best_av_p = av_p;
            best_epoch = epoch;
            
        if (av_p_all > best_av_p_all):
            best_model_all = clone_model(model_all);
            best_av_p_all = av_p_all;
            best_epoch_all = epoch_all;
    print()
    
    print('All:')
    print(best_av_p_all)
    print(best_epoch_all)

    predicted_val = best_model_all.predict(X_test)
    for pred, sentence in zip(predicted_val[0], test_sentences):
        sentence.pred = pred[0]

    print(average_precision(test_sentences,1))

    print('Pf:')
    print(best_av_p)
    print(best_epoch)

    predicted_val = best_model.predict(X_test)
    for pred, sentence in zip(predicted_val[1], test_sentences):
        sentence.pred = pred[0]

    print(average_precision(test_sentences,1,lambda x: x.labels[5]))
    

In [27]:
def run_model(model, train_target, label_func):
    # TRAINING
    # Train the model using the training set
    print("Start training model.")
    model.fit(X_train, train_target)
    print("Finished training model.")
    
    predicted_val = model.predict_proba(X_test)
    predicted_val = MinMaxScaler().fit_transform([pred for pred in predicted_val]).tolist()
    
    for pred, sentence in zip(predicted_val, test_sentences):
        sentence.pred = pred[1]
    print(average_precision(test_sentences,1,label_func))
    

In [35]:
run_single_model(y_train_pf, lambda x: x.label)

0:0.22783351048080033 1:0.21717950936927102 2:0.21782584130876145 3:0.2180520253620826 4:0.21957998222546332 5:0.22106181629721455 6:0.21975023133292895 7:0.22036916446118443 8:0.21859539486813745 9:0.22169165117337858 10:0.21854871577047671 11:0.21998636543591313 12:0.219772840067682 13:0.21810878869934194 14:0.2149346491667143 15:0.21798468488583522 16:0.21529664849126318 17:0.21616102449780294 18:0.21378485905373315 19:0.21563903543820903 
0.22783351048080033
5
0.17184869712609355


[<keras.engine.training.Model at 0x7f6509bc0f60>, 0.22783351048080033, 0]

In [ ]:
run_single_model(y_train_pf, lambda x: x.label)

In [16]:
run_multiple_times_single(y_train_pf, lambda x: x.labels[5])

0:0.0816799441951372 1:0.08480889010507392 2:0.08525218022332721 3:0.07758946600993591 4:0.0844291711697922 5:0.08341415795280815 6:0.07818655212537882 7:0.07671115523926116 8:0.08190802339913694 9:0.08470770018305825 10:0.08515789723657782 11:0.07542268062140244 12:0.08051275984754143 13:0.08150382764420472 14:0.07863176478502215 15:0.08465545255244532 16:0.0775146081416086 17:0.08099810763761844 18:0.08301271143075784 19:0.08331193101883295 20:0.08476300293706106 21:0.08917111365296283 22:0.08004106735791279 23:0.08236426282364799 24:0.08188588917690619 25:0.08078060784645574 26:0.07979509696366892 27:0.07842703531229252 28:0.07798613972520517 29:0.08293304438017518 30:0.08726534347072322 31:0.08014815637331683 32:0.07932395212092351 33:0.07651561112911343 34:0.08210354762727619 35:0.07800099515720017 36:0.07465203267469875 37:0.0781846635109224 38:0.07731945859675415 39:0.08074471844769239 40:0.08176625362040837 41:0.08359451598039189 42:0.07681083885763104 43:0.08218580003973006 44

In [17]:
run_multiple_times_single(y_train_all, lambda x: x.label)

0:0.2959923566855628 1:0.26777804253569104 2:0.2800670805392862 3:0.2943152364170195 4:0.2771883196099317 5:0.29125992391567523 6:0.2807235991877473 7:0.2731229584755554 8:0.2818831248244777 9:0.2896156620154064 10:0.2864594621946322 11:0.273708727278938 12:0.2868464879117518 13:0.2967489018827364 14:0.26791465731906383 15:0.28970405429800866 16:0.2858319032442378 17:0.27021193736486426 18:0.29153916890443465 19:0.27769087741672205 20:0.2648802271542685 21:0.27823168193520503 22:0.3003503814861282 23:0.2914966010388375 24:0.28248189385092065 25:0.2703913242151129 26:0.28352241738380835 27:0.27671430227634325 28:0.2938076822148246 29:0.28837354479782684 30:0.28664716543155316 31:0.276575495519297 32:0.2975828587814472 33:0.28458889737938114 34:0.2841966626914128 35:0.28061875178547563 36:0.29155606634888476 37:0.28266493447070185 38:0.2691509814122788 39:0.2667443647188395 40:0.2927821313360341 41:0.284829970668317 42:0.28681973691962204 43:0.2690549699438455 44:0.28930565968003863 45:0

In [ ]:
run_multiple_times_multiple([y_train_all, y_train_pf])

0:0.08476132129464271 0:0.3107075044291365 1:0.08558028898544857 1:0.290070129880259 2:0.08122200568448998 2:0.2958161539136481 3:0.07898196712028441 3:0.2748048626508349 4:0.08399431286900397 4:0.28752025976924384 5:0.08444418402970334 5:0.2947430872110886 6:0.09076107665826462 6:0.2979905027592441 7:0.08117146283232252 7:0.29025675683114033 8:0.08348613187177499 8:0.2875202195338798 9:0.08164491081485371 9:0.27987621383624584 10:0.08456899125978308 10:0.2938457355558174 11:0.0812880186338435 11:0.28701187845037257 12:0.08309202161545563 12:0.2734202648527543 13:0.0872137456180459 13:0.2744427295993051 14:0.08627683953377734 14:0.28202544422045156 15:0.08067430185346966 15:0.2903977735655574 16:0.08103911835787649 16:0.2861624352050825 17:0.08088987448659553 17:0.27026798995620005 18:0.08220503012739273 18:0.2833988789606126 19:0.07895470961932724 19:0.28457227542981356 20:0.0846209664109115 20:0.29432189713448953 21:0.09533205463829086 21:0.2888266052844089 22:0.08129304942164194 22:

In [17]:
run_multiple_times_multiple([y_train_all, y_train_0, y_train_1, y_train_2, y_train_3, y_train_4, y_train_5, y_train_6])


0:0.08465472306775067 0:0.2828367741616345 1:0.08903517326863744 1:0.3061784807867154 2:0.09118600649398148 2:0.2983773160270636 3:0.08416155336100359 3:0.3053478497708418 4:0.08602868290833388 4:0.27771351418942675 5:0.08938549379197616 5:0.28756284969755436 6:0.08573422333073279 6:0.28785556951147534 7:0.08811954822924103 7:0.28793380329939455 8:0.08559455444590641 8:0.2805937986750401 9:0.08892448202942416 9:0.29913398410963954 
All:
0.3061784807867154
2
0.20186961996098074
Pf:
0.09118600649398148
6
0.05005328844904463


In [30]:
run_model(SVC(class_weight='balanced', kernel='rbf', C=1, gamma=0.0001, probability=True, random_state=0),
        y_train_pf, lambda x: x.labels[5])

Start training model.
Finished training model.
0.03147209533439084


In [31]:
run_model(SVC(class_weight='balanced', kernel='rbf', C=1, gamma=0.0001, probability=True, random_state=0),
        y_train_all, lambda x: x.label)

Start training model.
Finished training model.
0.1275001023697541


In [28]:
 run_model(MLPClassifier(max_iter=100, solver='sgd', alpha=4, hidden_layer_sizes=(200, 50),
                            random_state=1, activation='relu', learning_rate_init=0.03, batch_size=550),
        y_train_pf, lambda x: x.labels[5])

Start training model.
Finished training model.
0.06109935407157336


In [29]:
 run_model(MLPClassifier(max_iter=100, solver='sgd', alpha=4, hidden_layer_sizes=(200, 50),
                            random_state=1, activation='relu', learning_rate_init=0.03, batch_size=550),
        y_train_all, lambda x: x.label)

Start training model.
Finished training model.
0.31253008848145997


In [ ]:
# class MicroMock(object):
#     def __init__(self, **kwargs):
#         self.__dict__.update(kwargs)


In [ ]:
# X_train = [[1,1,0.7],[0,1, 0.3]]
# X_test = [[0,0.9, 0],[1,0.3, 1]]
# y_train_pf = [1, 1]
# test_sentences = [MicroMock(label=0), MicroMock(label=1)]
# run_model(MLPClassifier(max_iter=10, solver='sgd', alpha=4, hidden_layer_sizes=(200, 50),
#                             random_state=1, activation='relu', learning_rate_init=0.03, batch_size=550),
#         y_train_pf, lambda x: x.label)